In [ ]:
import torch
from autoemulate.experimental.emulators import GaussianProcessExact

In [ ]:
from autoemulate.experimental.simulations.base import Simulator


class SignalGenerator(Simulator):
    def __init__(self, parameters_range = None, output_names = None, log_level = "progress_bar"):
        if parameters_range is None:
            parameters_range = {
                "phase": (-3.14, 3.14),
                "amplitude": (0, 1),
                "frequency": (0.1, 10)
            }
        if output_names is None:
            output_names = ["signal"]
        super().__init__(parameters_range, output_names, log_level)
    
    def _forward(self, x):
        phase = x[:, 0]
        amplitude = x[:, 1]
        frequency = x[:, 2]
        t = torch.linspace(0, 1, steps=100)
        signal = amplitude * torch.sin(2 * torch.pi * frequency * t + phase)
        return signal.unsqueeze(0)

In [ ]:
parameters_range = {
    "phase": (-3.14, 3.14),
    # "phase": (0, 0),
    # "amplitude": (1, 1.001),
    "amplitude": (0, 1),
    # Difficult to compress since all frequencies are present
    # "frequency": (1, 10),
    "frequency": (1, 1.0001)
}
sim = SignalGenerator(parameters_range)
n_samples = 200
x = sim.sample_inputs(n_samples)
y= sim.forward_batch(x)

In [ ]:
import matplotlib.pyplot as plt
for idx in range(5):
    plt.plot(y[idx], label=f"Signal {idx}")

In [ ]:
from autoemulate.experimental.compare import AutoEmulate
from autoemulate.experimental.transforms.discrete_fourier import DiscreteFourierTransform
from autoemulate.experimental.transforms import StandardizeTransform, PCATransform


ae = AutoEmulate(
    x,
    y,
    models=[GaussianProcessExact],
    x_transforms_list=[[]],
    # Can learn for phase and amplitude with small number of components
    y_transforms_list=[
        [PCATransform(n_components=1), StandardizeTransform()],
        [PCATransform(n_components=1)],
        [DiscreteFourierTransform(n_components=1), StandardizeTransform()],
        [DiscreteFourierTransform(n_components=1)],
        [PCATransform(n_components=2), StandardizeTransform()],
        [PCATransform(n_components=2)],
        [DiscreteFourierTransform(n_components=2), StandardizeTransform()],
        [DiscreteFourierTransform(n_components=2)],
    ],
    n_splits=5,
    log_level="progress_bar",
    tune=False
)

In [ ]:
ae.summarize()

In [ ]:
# Try varying frequency too
parameters_range = {
    "phase": (-3.14, 3.14),
    # "phase": (0, 0),
    # "amplitude": (1, 1.001),
    "amplitude": (0, 1),
    # Difficult to compress since all frequencies are present
    "frequency": (1, 10),
}
sim = SignalGenerator(parameters_range)
x = sim.sample_inputs(200)
y = sim.forward_batch(x)

ae = AutoEmulate(
    x,
    y,
    models=[GaussianProcessExact],
    x_transforms_list=[[]],
    # Can learn for phase and amplitude, but not frequency
    y_transforms_list=[[DiscreteFourierTransform(n_components=40)]],
    # y_transforms_list=[[DiscreteFourierTransform(n_components=20)]],
    n_splits=5,
    log_level="progress_bar",
    tune=False
)

In [ ]:
ae.summarize().T